# Тестовое задание от VK

## Импортируем библиотеки
> Пригодятся только для тестирования времени выполнения программы. В самой программе они не требуются.

In [125]:
import lorem # https://pypi.org/project/python-lorem/
import random

## Вводим используемые в проекте функции

### rounder_upper - окруление числа в большую сторону

На вход поступает `float` число, из которого выходит `int`, округленное в большую сторону
```
3.45 -> 4
5.7 -> 6
9.0 -> 9
```

In [127]:
def rounder_upper(num: float) -> int:
    "Округление float числа в большую сторону"
    int_part = int(str(num).split('.')[0])
    if num == int_part:
        return int(num)
    else:
        return int(num + (1-(num - int_part)))

### calculate_suffix_len - расчет длины суффикса в зависимости от количества секций в выходной строке

На вход поступает длина исходной строки (с учетом пробелов) в `int`. На выходе получаем необходимое количество символов, которые нужно оставить "про запас" в выходной строке.

In [135]:
def calculate_suffix_len(string_len: int) -> int:
    "Расчет длины суффикса в зависимости от количества секций"
    sections = rounder_upper(string_len/140) # получаем примерное количество секций, которое будет в результате работы нашего ПО
    if sections <= 9: # 9 секций дают следующий формат вывода: N/N - а это три символа. Именно такое количество мы и резервируем
        return 3
    elif sections > 9 and sections <= 99: # от 10 до 99 секций дают следующий формат вывода: NN/NN - а это пять символов
        return 5
    elif sections > 99 and sections <= 999: # от 100 до 999 секций дают следующий формат вывода: NNN/NNN - а это пять символов
        return 7
    else: # по ТЗ количество секций не может быть больше 9999, поэтому ограничимся 9 резервными символами - NNNN/NNNN
        return 9

#### Демонстрация работы

In [136]:
k = 1
print('string len -> number of sections -> suffix len')
while k !=10:
    ran = random.randint(30, 50000)
    print(f'{ran} -> {rounder_upper(ran/140)} -> {calculate_suffix_len(ran)}')
    k+=1

string len -> number of sections -> suffix len
34434 -> 246 -> 7
48095 -> 344 -> 7
21477 -> 154 -> 7
43007 -> 308 -> 7
16233 -> 116 -> 7
39539 -> 283 -> 7
9589 -> 69 -> 5
19929 -> 143 -> 7
42570 -> 305 -> 7


### line_separator - основная функция, выполняющая разделение строки на несколько секций.

ТЗ:

1. Функция принимает на вход текст, написанный латиницей, где все слова
разделены только 1 пробелом. Текст содержит в себе только латинские буквы и
пробелы (без знаков препинания).   
2. Задача функции разбить текст на СМСки размером не больше 140 символов и
вернуть в результате массив получившихся строк.   
3. Так как СМСки платные важно разбить текст на минимальное количество
фрагментов.   
4. Слова нельзя разбивать посередине, то есть текст нужно разбить строго по
пробелам.   
5. Если весь текст не помещается в один фрагмент то каждый фрагмент должен
заканчиваться суффиксом ' k/n', где k - порядковый номер фрагмента, n -
количество фрагментов, на которые разбит текст. k <= n <= 9999.   
6. Суффикс учитывается при подсчете длины СМСки, то есть длина фрагмента
вместе с суффиксом должна быть меньше или равна 140 символам.   
7. Задача решаема, то есть в тексте не может быть настолько длинных слов, чтобы
одно слово с суффиксом не уместилось в одном СМС.   

In [129]:
def line_separator(current_line: str) -> list:
    "Разделение строки на несколько блоков, содержащих максимум по 140 символов с добавлением суффикса"
    if len(current_line) <= 140:
        return [current_line]
    else: 
        total_len = 140 - calculate_suffix_len(len(current_line)) # получаем длину суффикса - именно такое количество символов должны мы зарезервировать из 140 символов
        sections_num = rounder_upper(len(current_line)/(total_len)) # считаем количество секций, которое будет в выходном сообщении
        split_line = current_line.split() # разделяем исходную строку по пробелу
        counter = 0 # добавляем счетчик и несколько пустых списков, для дальнейшей работы
        string = []
        total_string = []
        for element in split_line: # проходимся по всем элементам списка и проверяем их длину
            # если общая длина строки выходит за допустимую - объединяем получившуюся строку и продолжаем итерацию, очистив счетчик и временный список
            if counter+len(element) >= total_len: 
                counter = len(element)
                total_string.append(' '.join(string))
                string = [element]
            # иначе - продолжаем итерацию, до тех пор пока длина получившейся строки не достигает нужной длины
            else:
                counter+=len(element)
                string.append(element)
        total_string.append(' '.join(string)) # прибавляем в список последний получившийся элемент

        # теперь работаем с суффиксами - добавляем их к концу каждой строки
        counter = 1
        for i in range(0, len(total_string)):
            total_string[i] = total_string[i]+f' {counter}/{len(total_string)}'
            counter+=1
        return total_string

### Тесты

### Тест №1. Строка, содержащая до 140 символов.

In [142]:
line_separator(lorem.get_word(count=8))

['ut ex enim proident ullamco']

#### Замеры времени выполнения по заготовленной строке
> Для объективной оценки времени выполнения, воспользуемся заготовленной строкой

In [143]:
%%timeit
line_separator('Lorem ipsum dolor sit amet consectetur adipiscing elit')

888 ns ± 3.39 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


> Обратите внимание, в данном тесте используется подключаемая библиотека с генератором текста, что может замедлить выполнение программы

In [144]:
%%timeit
line_separator(lorem.get_word(count=8))

116 µs ± 948 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Тест №2. Строка, содержащая свыше 140 символов.

> Обратите внимание, в данном тесте используется подключаемая библиотека с генератором текста, что может замедлить выполнение программы

In [145]:
line_separator(lorem.get_word(count=random.randint(100, 120)))

['nisi amet ullamco voluptate excepteur consectetur incididunt dolor culpa ea consequat enim velit tempor enim eiusmod ex aliqua sed 1/7',
 'nulla eiusmod cupidatat irure incididunt lorem aute reprehenderit laboris non consequat nisi lorem occaecat non sit incididunt labore 2/7',
 'ullamco occaecat in cillum commodo cupidatat sed pariatur quis ut incididunt non elit lorem eu incididunt exercitation deserunt esse 3/7',
 'enim velit laborum incididunt occaecat aliquip ullamco sint mollit dolore quis adipiscing adipiscing et voluptate culpa deserunt minim 4/7',
 'voluptate aute ex cupidatat aute voluptate cupidatat occaecat elit fugiat exercitation culpa culpa et laborum incididunt aliqua eiusmod 5/7',
 'amet nostrud id sunt sed commodo laborum excepteur adipiscing cupidatat cillum pariatur dolore magna veniam eu pariatur ex consectetur 6/7',
 'consectetur qui 7/7']

#### Замер с использованием генератора слов

In [131]:
%%timeit
line_separator(lorem.get_word(count=random.randint(1000, 1200)))
# line_separator(sample_line)

15.9 ms ± 254 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [132]:
%%timeit
line_separator(sample_line)

355 µs ± 3.21 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
